 #                                      **IMAGE PROCESSING / OBJECT DETECTION**

> # Features Overview

The Cloud AutoML Vision Object Detection release includes the following features:

* **Object localization** - Detects multiple objects in an image and provides information about the object and where the object was found in the image.

* **API/UI** - Provides an API and custom user interface for importing your dataset from a Google Cloud Storage hosted CSV file and training images, for adding and removing annotations from imported images, for training and reviewing model evaluation metrics, and for using your model with online prediction.

AutoML Vision Edge now allows you to export your custom AutoML Vision Object Detection trained models.

* AutoML Vision Edge allows you to train and deploy low-latency, high accuracy models optimized for edge devices.
* With TensorFlow Lite, Core ML, and container export formats, AutoML Vision Edge supports a variety of devices.
* Hardware architectures supported: Edge TPUs, ARM and NVIDIA.
* To build an application on iOS or Android devices you can use AutoML Vision Edge in ML Kit. This solution is available via Firebase and offers an end-to-end development flow for
* creating and deploying custom models to mobile devices using ML Kit client libraries.

<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/index_test_image.png" alt="drawing" width="50%"/>

# Introduction:
This exploratory notebook aims to accomplish the following things
* get a feel for the images and to objects/segments they contain
* Look at image sizes
* Look at the frequency of objects
* Explore the hierarchy of objects in the dataset

In [ ]:
! conda install -y hvplot

In [ ]:
import os
import glob
import cv2
from pathlib import Path
import numpy as np
import pandas as pd
import hvplot.pandas
from dask import bag, diagnostics
import matplotlib.pyplot as plt

# Images and Annotations
Below are some examples of images with both boxes and masks. Note that the annotations and classes are different for the two competitions. The detection dataset has more classes of objects and more objects per images most of the time.

In [ ]:
data_dir = Path('../input/excerpt-from-openimages-2020-train')
im_list = sorted(data_dir.glob('train_00_part/*.jpg'))
mask_list = sorted(data_dir.glob('train-masks-f/*.png'))
boxes_df = pd.read_csv(data_dir/'oidv6-train-annotations-bbox.csv')

In [ ]:
im_ids = [im.stem for im in im_list]
cols = ['ImageID', 'LabelName','XMin','YMin','XMax', 'YMax']
boxes_df2 = boxes_df.loc[boxes_df.ImageID.isin(im_ids), cols]

names_ = ['LabelName', 'Label']
labels = pd.read_csv(data_dir/'class-descriptions-boxable.csv', names=names_)

boxes_df2 = boxes_df2.merge(labels, how='left', on='LabelName')

In [ ]:
cols, rows = 3,2
plt.figure(figsize=(20,30))

for i,im_file in enumerate(im_list[9:15], start=1):
    df = boxes_df2.query('ImageID == @im_file.stem').copy()
    img = cv2.imread(str(im_file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Add boxes
    h0, w0 = img.shape[:2]
    coords = ['XMin', 'YMin', 'XMax', 'YMax']
    df[coords] = (df[coords].to_numpy() * np.tile([w0, h0], 2)).astype(int)
    
    for tup in df.itertuples():
        cv2.rectangle(img, (tup.XMin, tup.YMin), (tup.XMax, tup.YMax),
                      color=(0,255,140), thickness=2)
        cv2.putText(img, tup.Label, (tup.XMin+2, tup.YMax-2),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=(255,0,0), thickness=2)
        
        #add segmentation mask
    mask_files = [m for m in mask_list if im_file.stem in m.stem]    
    mask_master = np.zeros_like(img)
    np.random.seed(10)
    for m in mask_files:
        mask = cv2.imread(str(m))
        mask = cv2.resize(mask, (w0,h0), interpolation = cv2.INTER_AREA)
        color = np.random.choice([0,255], size=3)
        mask[np.where((mask==[255, 255, 255]).all(axis=2))] = color
        mask_master = cv2.add(mask_master, 0.5, 0)
        
    img = cv2.addWeighted(img, 1, mask_master, 0.5, 0)
    
    #Plotting images
    plt.subplot(cols, rows, i)
    plt.axis('off')
    plt.imshow(img)
    
#print images 
plt.show()

In [ ]:
annotations = boxes_df.groupby('ImageID').agg(
    box_count =('LabelName', 'size'),
    box_unique= ('LabelName','nunique'))

pd.options.display.float_format = '{:,.1f}'.format
annotations.describe()

In [ ]:
annotations = boxes_df.groupby('ImageID').agg(
    box_count =('LabelName', 'size'),
    box_unique= ('LabelName','nunique'))

pd.options.display.float_format = '{:,.1f}'.format
annotations.describe()

In [ ]:
all = annotations.hvplot.hist('box_count',width=600, bins=30)
unique = annotations.hvplot.hist('box_unique', width=600)
(all + unique).cols(1)

Here's another look at the number of boxes per images with the largest 1% removed

In [ ]:
onepct = annotations.box_count.quantile(0.99)
annotations.query('box_count < @onepct').box_count.value_counts(normalize=True) \
.sort_index().hvplot.bar(xticks=list(range(0,60,10)),width=600,
                        line_alpha=0, xlabel='objects per image',
                        ylabel='fraction of image')

In [ ]:
print(boxes_df2.loc[boxes_df2.ImageID == 'fe7c6f7d298893da']\
     .groupby(['ImageID', 'Label'])['Label'].size())

im_file = "../input/excerpt-from-openimages-2020-train/train_00_part/fe7c6f7d298893da.jpg"
im = cv2.imread(im_file)
plt.imshow(im)

# Image Size

The dataset is huge! knowing image sizes can give an idea of the impact of size reduction. Here is now the test image sizes ae distributed.

In [ ]:
from PIL import Image
from dask import bag, diagnostics


def faster_get_dims(file):
    dims = Image.open(file).size
    return dims

dfile_list = glob.glob('../input/open-images-object-detection-rvc-2020/test/*.jpg')
print(f"Getting dimensions for {len(dfile_list)} files.")

# parallelize
dfile_bag = bag.from_sequence(dfile_list).map(faster_get_dims)
with diagnostics.ProgressBar():
    dims_list = dfile_bag.compute()

In [ ]:
sizes = pd.DataFrame(dims_list, columns = ['width', 'height'])
counts = sizes.groupby(['width', 'height']).agg(count=('width', 'size'))\
.reset_index()

In [ ]:
plot_opts = dict(xlim=(0,1200),
                 ylim = (0,1200),
                 grid=True,
                 xticks=[250,682,768,1024],
                 yticks=[250,682,768,1024],
                 height=500,
                 width=550
                )

style_opts = dict(scaling_factor=0.2,
                 line_alpha=1,
                 fill_alpha=0.1
                 )

counts.hvplot.scatter(x='width', y='height',size='count', **plot_opts).options(**style_opts)

# Distribution of Object labels
Here's a chart showing the frequency of the varouos types of objects. This is for detection, and for the train set, which will be different for instance segmentation and maybe for the test set. Overall thogh the datawill mostly be the pictures of 
* PEOPLE WITH FACES
* WEARING VLOTHES
* STANDING NEAR TREES, BUILDING
*

In [ ]:
train_labels = boxes_df[['ImageID','LabelName']].merge(labels,how='left',on='LabelName')
train_labels.Label.value_counts(normalize=True)[:45]\
.hvplot.bar(width=650, height=350, rot=60, line_alpha=0,
           title='Label Frequencies',
           ylabel = 'fraction of all objetcts')

In [ ]:
relations = pd.read_csv(data_dir/'oidv6-relationship-triplets.csv')
relations = relations.merge(labels, how='left', left_on='LabelName1', right_on='LabelName') \
                     .merge(labels, how='left', left_on='LabelName2', right_on='LabelName',
                            suffixes=['1', '2']) \
                     .loc[:, ['Label1', 'RelationshipLabel', 'Label2']] \
                     .dropna() \
                     .sort_values('RelationshipLabel') \
                     .reset_index(drop=True)

In [ ]:
import networkx as nx

kids = relations.query('Label1=="Girl" or Label1=="Boy"')
G = nx.from_pandas_edgelist(kids, 'Label1', 'Label2', 'RelationshipLabel')


graph_opts = dict(arrows=False,
                  node_size=5,
                  width=0.5,
                  alpha=0.8,
                  font_size=10,
                  font_color='darkblue',
                  edge_color='gray'
                
                 )

fig= plt.figure(figsize=(12,10))
nx.draw_spring(G, with_labels=True, **graph_opts)

# Thank You